# Multiscale template matching

When we try to apply template matching using the cv2.matchTemplate function, **usually the is the problem of false match** — this is because **the size of the image to match against the original image is smaller or bigger**

Given that the dimensions of the template image does not match the dimensions of of the image we want to match it to, we are left with a false detection.

> So what do we do now?

Apply a little trick:

- Loop over the input image at multiple scales (i.e. make the input image progressively smaller and smaller).

- Apply template matching using cv2.matchTemplate and keep track of the match with the largest correlation coefficient (along with the x, y-coordinates of the region with the largest correlation coefficient).

- After looping over all scales, take the region with the largest correlation coefficient and use that as your “matched” region.


```But template matching is not ideal if you are trying to match rotated objects or objects that exhibit non-affine transformations (Affine transforms preserve parallel lines). If you are concerned with these types of transformations you are better of jumping right to keypoint matching.```

In [7]:
import numpy as np
from imutils import paths
import imutils
import cv2 
import glob

In [8]:
#Load the image, grayscale, detect edges and get the height and width

template = cv2.imread('/media/juan/juan1/pyimage_univ/template_match/multiscale-template-matching/multiscale-template-matching/cod_logo.png')
template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
template = cv2.Canny(template, 50, 200)
(tH, tW) = template.shape[:2]

In [4]:
cv2.imshow("template", template)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
#loop trohugh all the images to search for the template in
for imagePath in glob.glob('/media/juan/juan1/pyimage_univ/template_match/multiscale-template-matching/multiscale-template-matching/images' + "/*.jpg"):
    #We do the same process as we did to the template image
    image= cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    found = None

    #Resize images at different scales
    #Create an array that has evenly series of numbers between 0.2 and 1. In this case 20 numbers
    for scale in np.linspace(0.2, 1.0, 20)[::-1]: #-1 so we go from big to small
        #resize
        resized = imutils.resize(gray, width= int(gray.shape[1]*scale))
        r = gray.shape[1] / float(resized.shape[1])

        #If the resized image is smaller than the template, break the loop
        if resized.shape[0] < tH or resized.shape[1] < tW:
            break

        #Detect the edges of the resized image. Grayscale it and apply template matching
        edged = cv2.Canny(resized, 50, 200)
        result = cv2.matchTemplate(edged, template, cv2.TM_CCOEFF)

        (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)

        #If we find a new maximum correlation value, update the variables

        # we update our bookkeeping variable found on Lines 61 and 62 to keep track of the maximum correlation value found thus far, 
        # the (x, y)-coordinate of the maximum value, along with the ratio of the original image width to the current, resized image width.
        if found is None or maxVal> found[0]:
            found = (maxVal, maxLoc, r)

    # unpack the variables and compute the (x, y) coordinates of the bounding box based 
    # on resized ratio, in order to resize the bounding boxes to the same scale of the 
    # original images
    (_, maxLoc, r) = found
    (startX, startY) = (int(maxLoc[0]*r), int(maxLoc[1]*r))
    #for the end points remember to add the original width and height
    (endX, endY) = (int((maxLoc[0] + tW)*r), int((maxLoc[1]+tH)*r))

    #draw the bounding boxes on the original images
    cv2.rectangle(image , (startX, startY), (endX, endY), (0,0,255), 2)
    cv2.imshow("image", image)
    cv2.waitKey(0)

cv2.destroyAllWindows()

[[  692516.25   627491.25   757540.75 ...   418761.    -946763.5
   -426563.5 ]
 [  818014.5   1013089.5    883039.5  ...   -70227.    -785502.
   -330327.25]
 [  471431.25   861580.75   991631.25 ...   303016.    -217183.75
    172966.  ]
 ...
 [ -881739.   -1206864.5   -296514.   ... -1195160.     170366.
    430464.5 ]
 [-1883124.   -1679596.2   -305618.5  ... -1650335.    -609934.5
   -219784.5 ]
 [-1491673.5  -1288145.2     94286.25 ... -1520285.5   -870034.5
   -154759.5 ]]
(137, 783) 8620364.0 (137, 783) (315, 403)
[[  -63074.25   171666.     462978.   ...  1417544.5   3540611.2
   1657487.2 ]
 [ -401205.25  -239942.75  -143705.75 ...  2025527.8   3433320.
   1680245.5 ]
 [ -340730.      15607.    -213281.   ...  2006021.2   3218737.5
   1530688.5 ]
 ...
 [-1940346.   -2061941.2   -818014.   ... -1251731.2   -418110.75
  -2244663.  ]
 [-2237509.8  -1960502.8   -838172.25 ... -1364875.2   -596279.25
  -2171184.8 ]
 [-1593112.   -1251081.5   -380396.25 ... -1096321.5   -912951.
  

In [22]:
 np.linspace(0.2, 0.1, 20)

array([0.2       , 0.19473684, 0.18947368, 0.18421053, 0.17894737,
       0.17368421, 0.16842105, 0.16315789, 0.15789474, 0.15263158,
       0.14736842, 0.14210526, 0.13684211, 0.13157895, 0.12631579,
       0.12105263, 0.11578947, 0.11052632, 0.10526316, 0.1       ])